## What does this do?

1. Bin all Gene Values
2. Sort users into training and test splits such that no more than 10 genes (out of 100) are from the same population (proven with t-test) --> see def similarity()

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from IPython.display import display, Markdown, Latex

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [8]:
import scipy.io
import pandas as pd
import numpy as np


mat = scipy.io.loadmat("/content/drive/MyDrive/BioInf/BRCA1View20000.mat")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
gene_labels = [g[0] for g in mat['gene'][0]]
df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
df['target'] = mat['targets']
df['Subtype'] = df.target.map({1:'Basal',2:'HER2+',3:'LumA',4:'LumB',5:'Normal Like',6:'Normal'})
df.head()

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,LOC642113,LOC652493,LOC647450,HLA-DRB1,LOC652694,SCGB2A1,HLA-A29.1,S100A9,CRABP1,S100A8,HMGCS2,ESR1,TCN1,KIAA1881,EEF1A2,COMP,LOC649923,CLEC3A,PROM1,LTF,SFRP1,ANKRD30A,RASD1,RERE,LOC441150,APOD,KRT15,FOSB,LOC651751,SCGB3A1,...,LOC651731,ITSN2,CT45-2,LOC283682,LOC650975,LOC644137,LOC653495,FBXO25,GSTM1L,MLANA,LOC653100,LOC256374,LOC649878,TTTY8,DAB1,LOC729967,LOC552891,LOC390332,C2orf19,LOC648365,LOC646867,LOC646861,NSL1,LOC645435,SYT5,C10orf62,LOC649942,LOC148003,DLEU7,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,target,Subtype
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,6.294989,6.050193,6.275648,5.353619,5.637666,5.444719,9.207865,8.020938,7.672754,8.179120,5.631433,7.209196,5.164125,5.330396,5.323875,6.107237,5.363248,5.730974,5.660094,5.987154,9.805175,5.825342,5.589548,5.239363,6.459878,12.885174,5.317683,10.907374,5.487921,5.436364,...,5.470603,5.545044,5.419480,5.617670,5.452332,5.256403,5.017035,5.475921,5.415913,5.058263,5.286056,5.236506,5.315590,5.326997,5.178206,5.351997,5.694721,5.300503,5.105706,5.244635,5.395235,5.446273,6.284404,5.386629,5.616339,5.245275,5.342450,5.361905,5.441128,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6,Normal
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,12.223921,11.990917,11.694773,10.544288,11.933992,10.942782,5.373724,7.052726,6.736109,7.918403,8.435546,11.245141,5.485468,11.795494,5.682812,6.335986,9.904679,5.143407,5.732892,5.666108,6.071107,6.708710,6.258828,6.107579,8.310777,10.974767,7.293080,6.640054,8.096736,5.934374,...,5.310653,5.455230,5.348173,5.133060,5.328521,5.323053,5.433345,5.554798,5.158888,5.220061,5.466661,5.331858,5.517638,5.234016,4.974619,5.412801,5.364871,5.535288,5.486211,5.563574,5.417604,5.221817,6.358165,5.493176,5.453547,5.450314,5.505897,5.393298,5.392657,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4,LumB
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,6.986047,6.598701,6.602586,5.383229,5.363445,12.690971,11.103585,7.020857,6.216404,6.589901,6.532649,11.886409,5.484449,6.720549,12.061691,6.337371,6.154567,6.212880,5.516034,5.626859,6.675468,6.405734,5.436449,6.368895,12.504543,10.821589,7.112686,7.927406,5.465041,5.434933,...,5.203277,5.371233,5.524234,5.468737,5.734440,5.560647,5.419495,5.849457,5.452299,5.245774,5.689215,5.231410,5.357202,5.428188,5.241753,5.680159,5.688863,5.944875,5.759466,5.348622,5.542033,5.521929,6.129600,5.406221,5.861127,5.319894,5.951688,5.278807,5.328035,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4,LumB
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,12.870259,12.147529,11.680246,5.464256,10.238677,10.113362,9.166876,9.256122,6.107621,12.208406,12.392228,10.762109,5.293256,8.076971,7.792335,9.885977,9.365163,5.688041,10.625052,5.352508,9.045812,10.473887,5.543993,11.666027,9.705837,9.037263,6.705601,10.387082,9.054395,6.371512,...,5.319892,5.319918,5.219195,5.119510,5.357346,5.066472,5.392240,5.501099,5.274038,5.431010,5.617247,5.056172,5.463829,5.386211,5.080487,5.235647,5.411676,5.256860,5.437603,5.315854,5.542615,5.459864,6.476763,5.493629,5.680635,5.369468,5.506951,5.422878,5.410289,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5,Normal Like
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,10.549090,12.127422,11.765410,10.733457,8.869752,8.416175,10.629661,8.119623,10.638175,8.418591,9.790858,11.684159,7.316475,10.793070,10.585061,10.262585,7.974055,7.727503,9.371779,6.873113,8.633202,9.638499,6.774566,9.252597,11.087252,13.345143,8.792931,6.542332,7.292459,5.975106,...,5.216493,5.541914,5.320086,5.139914,5.520602,5.506950,5.081318,5

In [10]:
lum_genes = pd.read_csv('Luminal_B_101_Gene_List.csv')

genes_df = df.loc[:,df.columns.isin(lum_genes["Gene"])]

FileNotFoundError: ignored

## Here is where we would select difference scalers

In [11]:
scaler = None

if scaler is None:
    scaler = StandardScaler()
    scaler.fit(genes_df)

genes_df_scaled = pd.DataFrame(scaler.transform(genes_df),index=genes_df.index,columns=genes_df.columns).fillna(0)

NameError: ignored

In [ ]:
genes_df_scaled.head()

,CCND1,FAM83H,C17orf28,FAM83D,SRGAP1,UBE2C,BOP1,BRF2,KIFC2,TROAP,ATAD2,C8orf55,PPAPDC1B,GPR172A,ASH2L,EVPL,EPN3,COX6C,LSM1,MTDH,GRHL2,STX1A,FBXL6,MYST3,PPP1R16A,SLC35B1,RECQL4,CPSF1,AP3M2,EIF4EBP1,TMEM97,RAD9A,RAB11FIP1,VDAC3,CTTN,ZC3H3,C20orf20,MTERFD1,ANKRD46,SLC25A39,...,INTS2,HSF1,E2F7,SHARPIN,YTHDF1,NUP107,CYHR1,TLK2,ADCK5,MPP3,TOP1MT,ATP2A2,CABLES2,CCNE2,SPAG5,ZNF703,GINS4,PYCRL,RRM2B,ESPL1,CYB561,TUBG1,GNPNAT1,EME1,SCRIB,ATP6V1C1,STX16,SLC39A4,TERF1,AGPAT6,RAD54B,NFKBIL2,ZBTB10,PSMC3IP,LETM2,BRCA1,FAM82B,MYBL2,FRS2,CIT
0,-0.100659,-3.189314,-1.163106,-1.368472,0.197607,-2.160346,-1.268345,0.235924,-0.933807,-1.898709,-1.349091,-0.957618,-1.324860,-1.212640,0.088892,-2.936120,-1.688567,-0.804162,-0.456540,0.557704,-2.944221,-1.610955,-1.847162,0.206467,1.250960,-1.196235,-1.530791,-1.052373,-0.627843,-0.015980,-2.028178,-0.458750,-1.308824,-0.695537,-0.561794,-1.787248,-2.167764,-0.063759,2.292573,-1.608163,...,0.440818,0.321424,-1.425527,-0.287942,-1.635358,-0.021086,-0.507660,-0.749560,-1.169973,0.707973,-1.637904,-0.332030,-1.886062,-1.056666,-0.100155,-0.804692,-1.237288,-1.684516,0.095985,-0.627755,-1.424847,-0.044642,-0.987478,-0.778012,-0.659475,-0.429621,0.151839,-1.035374,-0.693535,-2.048216,-1.410917,-0.539807,-0.990863,-1.394209,-1.683707,-1.760110,0.504558,-0.916205,-1.437369,1.309344
1,-0.396379,1.923176,0.505856,-0.638042,1.477128,0.040077,2.057732,2.492817,2.267095,-0.076725,1.074563,1.621953,-1.039426,2.038814,-0.754791,-0.158287,1.748301,0.439154,-0.948139,1.470209,1.518695,-0.038598,2.194620,3.246274,1.338708,3.587559,1.361066,1.328042,2.435652,-0.394550,-0.185640,0.198679,-1.029280,1.639654,-0.303790,2.661053,-0.990109,1.455086,1.104848,-0.671493,...,0.968933,0.979152,-1.110542,2.844377,-0.389303,-0.387586,2.615740,1.359727,2.713365,-1.395687,1.590122,0.418761,-1.438529,0.626745,2.667167,2.266990,1.050673,3.105571,1.032738,0.444366,1.627227,-0.950717,-0.194819,1.485297,2.416406,1.771702,-1.095261,3.747004,1.456115,1.137624,0.275690,0.731575,-0.143936,-0.736753,-0.378679,-0.281442,1.326211,1.086842,-0.746088,-0.877594
2,0.153928,2.076008,0.727836,-0.948505,0.592500,-0.527023,1.303371,0.617340,1.742926,-0.027667,0.337773,1.247554,0.625397,1.586047,-0.479971,0.969097,0.374611,0.712519,-0.664636,0.018087,0.580629,3.101621,1.077745,1.168013,0.430166,-0.563047,0.291943,-0.484993,0.086237,-1.127188,0.465257,0.557245,-0.270660,0.186048,0.606822,0.593375,-0.553368,-0.641475,2.532736,0.425729,...,-0.319292,2.095369,-1.022540,2.138964,0.400239,-1.529085,2.671968,-0.471980,1.234853,1.136127,2.751111,-0.113250,-1.858571,0.006061,-1.376185,0.496754,-1.236193,1.860988,-0.056248,0.024681,0.062876,-0.180601,0.825097,-1.486632,1.670547,-0.306558,-0.779561,1.264107,2.916525,-1.253905,-0.095986,-1.137496,-1.343864,-0.062959,0.889682,1.095956,2.489593,0.528456,0.021510,-0.872164
3,0.764704,0.830027,-0.425847,-0.142911,0.458682,0.967930,1.843483,-0.298788,1.120019,1.767628,-0.208164,-0.098424,-0.756544,1.162151,-0.570237,1.771165,-0.738980,-0.420044,0.598380,-0.344563,-0.197144,0.350060,1.030048,0.381792,0.559060,-0.902092,1.660105,1.674757,0.796356,-0.082314,1.014561,1.240019,-0.669726,-0.552967,1.036199,1.289725,0.555577,-0.973785,-1.679948,0.747850,...,-0.553737,-0.517851,0.515954,-0.089844,-0.633599,0.677793,-0.300918,-0.592295,1.560329,0.461802,0.401979,-0.596686,-0.320608,0.262063,0.467094,-0.171209,-0.025101,0.044943,-0.683678,1.495754,-0.617707,-0.622389,0.045590,-0.305385,1.366591,-0.871612,0.090735,0.682875,-0.692517,-0.505684,-0.692801,1.483901,-1.242918,0.485040,-0.336100,1.149192,0.498185,-1.617987,0.403393,2.220044
4,1.547597,0.188069,1.024349,-0.520472,0.484273,0.318252,-0.363769,2.339653,0.595944,0.124801,-1.002283,-0.422653,1.581828,0.182421,1.581736,1.281019,1.668517,0.515580,1.408567,0.197036,0.520945,0.059763,0.150591,0.797743,-0.277463,0.686857,-0.254308,-0.944257,1.121482,1.297535,-0.079958,1.501340,1.732626,-0.383915,0.579327,0.053487,0.213881,0.163847,-0.220036,0.945769,...,1.530554,-1.485307,-0.5826

In [ ]:
# Try Binning to Generate Unequal Training and Test Distributions

# Test Size of 10%
test_size = 200 #10%

In [ ]:
# Helper function to check how many of genes are from different populations

from scipy import stats 
def similarity(df1, df2):
    #print(stats.ttest_ind(df1, df2))
    return len([i for i in stats.ttest_ind(df1, df2).pvalue if i < 0.05])

In [ ]:
# Bin each Gene

binned = pd.qcut(genes_df_scaled.values.reshape(191970), 10, retbins=True)[1]
binned = [-100] + list(binned[1:-1]) + [100]
genes_df_scaled_binned = pd.DataFrame()
for i in genes_df_scaled.columns:
    genes_df_scaled_binned[i] = pd.cut(genes_df_scaled[i], bins=binned).astype(str)

In [ ]:
import itertools 

d = {}

for a in list(set(genes_df_scaled_binned.values.reshape(191970))):
    selected = (genes_df_scaled_binned == a).sum(axis=1).sort_values(ascending=False)[0:test_size]
    if(selected.values[-1] == 0):
        print(selected)
    test_ids = selected.index
    
    # If number of genes from different populations is close to 90, 
    # we can ensure the test set is different enough from the training set 

    s = similarity(genes_df_scaled.iloc[test_ids], genes_df_scaled.iloc[~genes_df_scaled.index.isin(test_ids)])
    if (s > 80):
        train_ids = [i for i in genes_df_scaled.index if i not in test_ids]
        d[a] = {'train_ids': list(train_ids), 'test_ids': list(test_ids)}
        


In [ ]:
import json
with open("shiftedsets.json", "w") as outfile: 
    json.dump(d, outfile)

In [ ]:

import json

with open('shiftedsets.json') as f:
  data = json.load(f)

In [ ]:
import random 
n = 0

for k, v in data.items():
  curr = pd.DataFrame(index=range(2133),columns=["train", "test", "val"])
  val = random.sample(v['test_ids'], 20)
  test = [i for i in v['test_ids'] if i not in val]
  train = v['train_ids']
  #print(len(val))
  #print(len(test))
  print(len(train))
  break
  for i in range(2133):
    curr.at[i, 'val'] = [True if i in val else False][0]
    curr.at[i, 'test'] = [True if i in test else False][0]
    curr.at[i, 'train'] = [True if i in train else False][0]
  print(n)
  curr.to_csv(f'split_{n}.csv')
  n += 1

1933


In [ ]:
curr[curr['test'] == True]

,train,test,val
8,False,True,False
10,False,True,False
12,False,True,False
21,False,True,False
23,False,True,False
...,...,...,...
2029,False,True,False
2105,False,True,False
2107,False,True,False
2109,False,True,False


In [ ]:
!mkdir splits

In [ ]:
!mv split* splits

mv: cannot move 'splits' to a subdirectory of itself, 'splits/splits'


In [ ]:
!zip -r splits.zip splits

updating: splits/ (stored 0%)
  adding: splits/split_2.csv (deflated 88%)
  adding: splits/split_0.csv (deflated 88%)
  adding: splits/split_5.csv (deflated 88%)
  adding: splits/split_6.csv (deflated 88%)
  adding: splits/split_1.csv (deflated 88%)
  adding: splits/split_4.csv (deflated 88%)
  adding: splits/split_3.csv (deflated 88%)
